In [1]:
import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import word_tokenize
import re
import matplotlib.pyplot as plt
import math
from nltk.corpus import stopwords
import collections
import spacy
from datetime import datetime
nlp = spacy.load("en_core_web_lg")

In [2]:
def reduce(l):
    sorted_reduced = []
    for i in l:
        if i not in sorted_reduced:
                sorted_reduced.append(i)

    return sorted_reduced

In [3]:
# skills 
final_skills = pd.read_csv(r"C:\Users\Dima\Desktop\FeatFiles\clusters\final clusters.csv")
hardsk=[]

softsk=list(final_skills["0"].dropna().values)

for col in final_skills.columns[1:]:  
    hardsk+= final_skills[col].dropna().to_list()
    
    
hardsk=list(map(lambda x: x.lower(), hardsk))
hardsk = reduce(hardsk)

softsk=list(map(lambda x: x.lower(), softsk))
softsk=reduce(softsk)


In [4]:
#text preprocessing
Data = pd.read_csv("JobDataTranslated.csv")
#Data['Job Description'].apply(lambda x: x.lower())

def clean(text):
    
    # removing paragraph numbers
    text = re.sub('[0-9]+.\t','',str(text))
    # removing new line characters
    text = re.sub('\n ','',str(text))
    text = re.sub('\n',' ',str(text))
    # removing apostrophes
    
    # removing hyphens
    text = re.sub("-",' ',str(text))
    text = re.sub("— ",'',str(text))
    # removing quotation marks
    text = re.sub('\"','',str(text))
    # removing salutations
    text = re.sub("Mr\.",'Mr',str(text))
    text = re.sub("Mrs\.",'Mrs',str(text))
    # removing any reference to outside text
    text = re.sub("[\(\[].*?[\)\]]", "", str(text))
    
    return text

# preprocessing speeches
Data['Cleaned Job Description'] = Data['Job Description'].apply(clean)

from string import punctuation

def remove_punctuation(s):
    global punctuation
    for p in punctuation:
        s = s.replace(p, '')
    return s

Data['Cleaned Job Description'] = Data['Cleaned Job Description'].map(remove_punctuation)

Data['Cleaned Job Description'] = Data['Cleaned Job Description'].apply(lambda x: x.lower())
 
#nltk.download('stopwords')
en_stopwords = stopwords.words('english')
def remove_stopwords(s):
    global en_stopwords
    s = word_tokenize(s)
    s = " ".join([w for w in s if w not in en_stopwords])
    return s

# Create a new column of descriptions with no stopwords
Data['Cleaned Job Description NoStop'] = Data['Cleaned Job Description'].map(remove_stopwords)





In [5]:
oldie =pd.read_pickle('first_tr_data.pkl') # this is datates from 'Word Analysis' notebook

In [15]:
# the only actual extraction part


def get_vac_tokens(vac):
    singles=[]
    doubles=[]
    vac_list=vac.split(' ')
    for i in vac_list:
        singles.append(nlp(i))
        
    for i in range(len(vac_list)-1):
        doubles.append(nlp(vac_list[i]+' ' + vac_list[i+1]))
    
    return singles, doubles
    


#vac - vacancy text, num - index
def get_top_skills(vac, skills,num):
    singles, doubles = get_vac_tokens(vac)
    
    l_single=[]
    l_double=[]
    
    two_words= [i for i in skills if i.count(' ')>0]
    one_words=[i for i in skills if i not in two_words]
    
    
    two_words=list(map(lambda x: nlp(x), two_words))
    one_words=list(map(lambda x: nlp(x), one_words))
   
    for i in one_words:
           for token2 in singles:
                l_single.append([i.similarity(token2), i, token2.text])
     
    for i in two_words:
           for token2 in doubles:
                 l_double.append([i.similarity(token2), i, token2.text])
                    
                    
    #this part is using fisrt dataset in new framework
    z=oldie[oldie[1]==num].values[0][0]

    z=list(map(lambda x: nlp(x), z))
    
    
    for i in two_words:
        for j in z:
            l_double.append([i.similarity(j), str(i),str(j)])
    
    for i in one_words:
        for j in z:
            l_single.append([i.similarity(j), str(i),str(j)])
                
    return l_single,l_double

In [18]:
def get_clusters(single,double,min_max=[3,10]):
    
    single=[i for i in single if float(i[0])>0.5]
    single=[[i[0],str(i[1]),i[2]] for i in single]
    double=[i for i in double if float(i[0])>0.5]
    double=[[i[0],str(i[1]),i[2]] for i in double]
    

    possible_clusters_single = reduce([i[1] for i in single])
    scores = np.zeros(len(possible_clusters_single))
    for i in single:
        scores[possible_clusters_single.index(i[1])]+=i[0]**3+2*max(np.sign(i[0]-0.9),0)
        
    scores_cl = [[scores[i], possible_clusters_single[i]] for i in range(len(scores))]
    scores_cl=sorted(scores_cl, reverse = True)
    
    
    possible_clusters_double = reduce([i[1] for i in double])
    scores = np.zeros(len(possible_clusters_double))
    for i in double:
        scores[possible_clusters_double.index(i[1])]+=i[0]**4+2*max(np.sign(i[0]-0.8),0)
        
    scores_cl +=[[scores[i], possible_clusters_double[i]] for i in range(len(scores))]
    scores_cl =sorted(scores_cl, reverse = True)
    
    
    

    cl=[scores_cl[i][1] for i in range(min_max[0])]
    for i in range(min_max[0],min_max[1]):
        if scores_cl[i][0]<2:
            break
        cl.append(scores_cl[i][1])
        
    return cl

In [25]:
def give_skills(num):
    

    single,double = get_top_skills(Data['Cleaned Job Description NoStop'][num], hardsk,num)
   
    s,d = get_top_skills(Data['Cleaned Job Description NoStop'][num], softsk,num)


  
    return get_clusters(single,double), get_clusters(s,d,min_max =[2,5])

In [19]:
give_skills(900)

C:\Users\Dima\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
C:\Users\Dima\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


0:00:09.530714


(['maintenance',
  'engineering',
  'management',
  'sales operations',
  'installation',
  'technician',
  'product testing',
  'healthcare',
  'repair',
  'construction'],
 ['business management',
  'critical thinking',
  'public speaking',
  'problem solving',
  'logical thinking'])

In [20]:
give_skills(100)

C:\Users\Dima\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
C:\Users\Dima\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


0:00:05.911202


(['mechanic',
  'installation',
  'technician',
  'engineering',
  'construction',
  'management',
  'product testing',
  'sales operations',
  'maintenance'],
 ['critical thinking',
  'business management',
  'public speaking',
  'leadership',
  'logical thinking'])

C:\Users\Dima\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
C:\Users\Dima\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


2

In [29]:
startTime = datetime.now()
super_sk_list=[]
for i in range(len(Data)):
    try:
        super_sk_list.append(give_skills(i))
    except IndexError:
        super_sk_list.append([-1,-1])
print(datetime.now() - startTime)

C:\Users\Dima\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
C:\Users\Dima\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
C:\Users\Dima\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


2:08:38.538078


In [32]:
import pickle
with open('super_skill_list.pkl', 'wb') as f:
    pickle.dump(super_sk_list, f)

ValueError: binary mode doesn't take an encoding argument